In [ ]:
## Les imports

from google.colab import drive
drive.mount('/content/drive')

import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
import pandas as pd
import csv
import re
from nltk.corpus import stopwords
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import warnings
warnings.filterwarnings("ignore")
import os
for dirname, _, filenames in os.walk('/Users/benadem/Desktop/4IABD/4IADB_DL'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
## Charger les jeux de données
%%time
dftr = pd.read_csv("/content/drive/MyDrive/goodreads_train.csv")
dfte = pd.read_csv("/content/drive/MyDrive/goodreads_test.csv")

In [ ]:
## Affichage des données

dfre = dftr[["review_text"]]
dfret = dfte[["review_text"]]
text = pd.concat([dfre,dfret])
text.shape
text

In [ ]:
## Supprimer les "Stopwords"

from nltk.corpus import stopwords
nltk.download('stopwords')

stop = set(stopwords.words("english"))
def remove_stopwords(text):
  text = [word.lower() for word in text.split() if word.lower() not in stop]
  return " ".join(text)

text["review_text"] = text["review_text"].map(remove_stopwords)

In [ ]:
%%time
## Commencer le traitement de la donnée

text["review_text"] = text["review_text"].map(remove_stopwords)
maxlen = 500
max_words = 10000
tokenizer = Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(text.review_text)
trainsequences = tokenizer.texts_to_sequences(dfre.review_text)
testsequences = tokenizer.texts_to_sequences(dfret.review_text)

In [ ]:
%%time
word_index = tokenizer.word_index
train = pad_sequences(trainsequences,padding = 'post', maxlen=500 )
test = pad_sequences(testsequences,padding = 'post', maxlen=500)
labels = ohe.fit_transform(dftr[["rating"]]).toarray()
print('Found %s unique tokens.' % len(word_index))
print('Shape of train data:', train.shape)
print("Shape of test data:",test.shape)
print('Shape of label tensor:', labels.shape)

In [ ]:
## Le Modele

from keras.models import Sequential
from keras.layers import Dense,Dropout,Embedding,Bidirectional,LSTM
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.regularizers import l2
cp = ModelCheckpoint('model/', save_best_only=True)

model = Sequential()
model.add(Embedding(10000,64))
model.add(Bidirectional(LSTM(128,dropout = 0.3,return_sequences=True)))
model.add(Bidirectional(LSTM(64,dropout = 0.3)))
model.add(Dense(64,activation='relu',kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(6,activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer=Adam(0.005), metrics=['accuracy'])
model.summary()

In [ ]:
## Tensorboard
from keras.callbacks import EarlyStopping,TensorBoard
import datetime
#cb1 = EarlyStopping(monitor='val_loss',patience=10,mode='min')
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
cb2 = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [ ]:
# Entrainement du modele
model.fit(train,labels,epochs=30,batch_size=1000,callbacks=[cb2],validation_split=0.2)

In [ ]:
pred = model.predict(test)

In [ ]:
pred = [np.argmax(a) for a in pred]

In [ ]:
ss = pd.read_csv("/content/drive/MyDrive/goodreads_sample_submission.csv")

In [ ]:
ss.rating = pred

In [ ]:
ss.to_csv("/content/drive/MyDrive/LSTMSubmission.csv",index=False)
%load_ext tensorboard
%tensorboard --logdir logs/fit